In [5]:
from pprint import pprint
from data_algebra.data_ops import *
import data_algebra.env
import data_algebra.yaml
import data_algebra.PostgreSQL

db_model = data_algebra.PostgreSQL.PostgreSQLModel()

_, _1, _2, _get = [None, None, None, lambda x: x] # don't look unbound
data_algebra.env.push_onto_namespace_stack(locals())

t1 = TableDescription('t1', ['x', 'y'])
print(t1.pretty_sql(db_model))

SELECT "x",
       "y"
FROM "t1"


In [ ]:
t1b = TableDescription('t1', ['x', 'y'])
t2 = TableDescription('t2', ['x', 'z'])

ops = ( 
        t1 >> 
            Extend({'x':'x + 1'}) >> 
            NaturalJoin(
                b=t1b,
                by=['x', 'y']) >> 
            NaturalJoin(
                b=(t2 >> Extend({'f':'x + 1'})),
                by='x') 
)
print(ops)

In [2]:
p = ops.collect_representation()
pprint(p)

[OrderedDict([('op', 'TableDescription'),
              ('table_name', 't1'),
              ('qualifiers', {}),
              ('column_names', ['x', 'y'])]),
 OrderedDict([('op', 'Extend'), ('ops', {'x': '(x + 1)'})]),
 OrderedDict([('op', 'NaturalJoin'),
              ('by', ['x', 'y']),
              ('jointype', 'INNER'),
              ('b',
               [OrderedDict([('op', 'TableDescription'),
                             ('table_name', 't1'),
                             ('qualifiers', {}),
                             ('column_names', ['x', 'y'])])])]),
 OrderedDict([('op', 'NaturalJoin'),
              ('by', ['x']),
              ('jointype', 'INNER'),
              ('b',
               [OrderedDict([('op', 'TableDescription'),
                             ('table_name', 't2'),
                             ('qualifiers', {}),
                             ('column_names', ['x', 'z'])]),
                OrderedDict([('op', 'Extend'), ('ops', {'f': '(x + 1)'})])])])]


In [3]:
import yaml
dmp = yaml.dump(p)
print(dmp)

- op: TableDescription
  table_name: t1
  qualifiers: {}
  column_names:
  - x
  - y
- op: Extend
  ops:
    x: (x + 1)
- op: NaturalJoin
  by:
  - x
  - y
  jointype: INNER
  b:
  - op: TableDescription
    table_name: t1
    qualifiers: {}
    column_names:
    - x
    - y
- op: NaturalJoin
  by:
  - x
  jointype: INNER
  b:
  - op: TableDescription
    table_name: t2
    qualifiers: {}
    column_names:
    - x
    - z
  - op: Extend
    ops:
      f: (x + 1)



In [4]:
ops_back = data_algebra.yaml.to_pipeline(yaml.safe_load(dmp))
print(ops_back)


t1 >>
   Extend({'x': (x + 1)}) >>
   NaturalJoin(b=(
      t1),
      by=['x', 'y'], jointype=INNER) >>
   NaturalJoin(b=(
      t2 >>
         Extend({'f': (x + 1)})),
      by=['x'], jointype=INNER)
